# 0.01 Function

In [1]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")
bound = 1e-4
error = 0.15
#theta1 = np.array([[-100,-1,10],[10,-1,-100]]) #s,a1,a2 #np.array([2,-1])#np.array([[-1,-1,1],[1,-1,-1]])#
#theta23 = np.array([[0.5,0.5,1-bound,bound,0.5,0.5],[0.5,0.5,1-error,error,0.5,0.5]])
theta1 = np.array([-5,-1,2]) #s,a1,a2 #np.array([2,-1])#np.array([[-1,-1,1],[1,-1,-1]])#
theta23 = np.array([[0.5,1-bound,0.5],[0.5,1-error,0.5]])

TimePeriod= 150
SamplePath = 5000
num_discrete = 100
Nsub = 100
alpha = 1

z_old = np.arange(2,dtype=int) #observation
a1_old = np.arange(3,dtype = int) # 0:left 1:listen 2:right
x_old = np.linspace(0,1,num_discrete) #belief
def Dynamic(theta23):
  theta23 = np.reshape(theta23,[2,3])
  trans = np.ones([2,3,2])      #s_t,a1,s_{t+1}
  trans[:,0,0] = [theta23[0,0],1-theta23[0,0]]#np.reshape(theta23[0:4],[2,2])
  trans[:,1,0] = [theta23[0,1],1-theta23[0,1]]#1 - trans[:,:,0]
  trans[:,2,0] = [theta23[0,2],1-theta23[0,2]]
  trans[:,:,1] = 1 - trans[:,:,0]
  observe = np.ones([3,2,2]) #a1,s_{t+1},z_{t+1}
  observe[0,:,0] = [theta23[1,0],1-theta23[1,0]]
  observe[1,:,0] = [theta23[1,1],1-theta23[1,1]]
  observe[2,:,0] = [theta23[1,2],1-theta23[1,2]]
  observe[:,:,1] = 1-observe[:,:,0] 
  return trans,observe
def Reward(theta1):
  rw = np.ones([2,3])#s,a1
  rw[0,0] = theta1[0]
  rw[1,2] = theta1[0]
  rw[0,1] = theta1[1]
  rw[1,1] = theta1[1]
  rw[0,2] = theta1[2]
  rw[1,0] = theta1[2]
  # rw[0,0] = 100* rw[0,0]
  # rw[1,2] = 100* rw[1,2]
  # rw[0,2] = 10 * rw[0,2]
  # rw[1,1] = 10 * rw[1,1]
  return rw
# def Dynamic(theta23):
#   theta23 = np.reshape(theta23,[2,6])
#   trans = np.ones([2,3,2])      #s_t,a1,s_{t+1}
#   trans[:,0,0] = theta23[0,0:2]#np.reshape(theta23[0:4],[2,2])
#   trans[:,1,0] = theta23[0,2:4]#1 - trans[:,:,0]
#   trans[:,2,0] = theta23[0,4:6]
#   trans[:,:,1] = 1 - trans[:,:,0]
#   observe = np.ones([3,2,2]) #a1,s_{t+1},z_{t+1}
#   observe[0,:,0] = theta23[1,0:2]
#   observe[1,:,0] = theta23[1,2:4]
#   observe[2,:,0] = theta23[1,4:6]
#   observe[:,:,1] = 1-observe[:,:,0] 
#   return trans,observe
#print(Dynamic(theta23))
# def Reward(theta1):
#   #rw = np.ones([2,3])#s,a1
#   rw = theta1.copy()
#   # rw[0,0] = 100* rw[0,0]
#   # rw[1,2] = 100* rw[1,2]
#   # rw[0,2] = 10 * rw[0,2]
#   # rw[1,1] = 10 * rw[1,1]
#   return rw
  
# print(Reward(theta1))
def SigmaLambda(z_old,x_old,a1_old,theta23,num_discrete=num_discrete,T = None): #xt,a1_t,z_{t+1}
  trans,observe = Dynamic(theta23)  #s_t,a1,s_{t+1} #a1,s_{t+1},z_{t+1}
  if T==None: #generate function for Q function iteration
      x_new = np.zeros([num_discrete,3,2])
      sigma = np.zeros([num_discrete,3,2])
      x_old = np.vstack([x_old,1-x_old]).T
      for a1 in a1_old:#range(2):
        for z_new in z_old:#range(2):
          x_new_tep = x_old.dot(trans[:,a1,0]*observe[a1,0,z_new])
          sigma[:,a1,z_new] = x_new_tep + x_old.dot(trans[:,a1,1]*observe[a1,1,z_new])
          x_new[:,a1,z_new] = x_new_tep/sigma[:,a1,z_new]
      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
  elif T==1: #generate 1 period data for data generation
      x_new = np.zeros(x_old.shape)
      sigma = np.zeros(x_old.shape)
      #x_old = np.vstack([x_old,1-x_old]).T
      x_new_tep = (x_old *trans[0,a1_old,0] + (1-x_old)*trans[1,a1_old,0] )* observe[a1_old,0,z_old]
      sigma = x_new_tep + (x_old *trans[0,a1_old,1] + (1-x_old)*trans[1,a1_old,1] )* observe[a1_old,1,z_old]
      x_new = x_new_tep/sigma
      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
  else:      #generate whole belief for recover process
      x_new = np.zeros(a1_old.shape) # T, N
      sigma = np.zeros([T-1,z_old.shape[1]]) # T-1 ,N
      x_new[0,:] = x_old
      #x_old = np.vstack([x_old,1-x_old]).T
      for t in range(1,T):
        a1 = a1_old[t-1,:]
        z = z_old[t,:]
        x_new_tep = (x_new[t-1,:] *trans[0,a1,0] + (1-x_new[t-1,:])*trans[1,a1,0] )* observe[a1,0,z]
        sigma[t-1,:] = x_new_tep + (x_new[t-1,:] *trans[0,a1,1] + (1-x_new[t-1,:])*trans[1,a1,1] )* observe[a1,1,z]
        x_new[t,:] = x_new_tep/sigma[t-1,:]
      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  return sigma,x_new,[iterpolate,np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))]
#print(SigmaLambda(z_old,x_old,a1_old,theta23)[0])

def ValueIteration(theta1,sigma,xiterpolate, num_discrete=num_discrete,alpha=1,beta = 0.95,
                   gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5): #xt, a1 or xt,a2
  rw = Reward(theta1)
  iterpolate,belief_f,belief_c = xiterpolate[0],xiterpolate[1],xiterpolate[2]
  x_old = np.linspace(0,1,num=num_discrete)
  pi1_new = 1/3*np.ones([num_discrete,3]) #xt,a1
  Q1_new = np.zeros([num_discrete,3]) #xt,a2


  for il in range(1000):
    Q1_old = Q1_new.copy()
    pi1_old = pi1_new.copy()
    for a1 in range(3):
      #a2 = 0
      it0,bf0,bc0 = iterpolate[:,a1,:],belief_f[:,a1,:],belief_c[:,a1,:]
      V1_a20 =  np.stack([it0*Q1_old[bf0,0] + (1-it0)*Q1_old[bc0,0], it0*Q1_old[bf0,1] + (1-it0)*Q1_old[bc0,1]],axis=2)/alpha
      Q1_new[:,a1] = x_old * rw[0,a1] + (1-x_old) * rw[1,a1] + beta * np.sum(sigma[:,a1,:]*(gamma+logsumexp(V1_a20,axis=2))*alpha,axis=1)
        

    pi1_new = softmax(Q1_new/alpha,axis=1)
    #pi1_new[np.where(pi1_new<=bound)] = 0
    #print(il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)),np.max(np.abs(Q1_old)),np.max(np.abs((pi1_new-pi1_old)/pi1_old)),np.max(np.abs(pi1_old)))
    if np.max(np.abs((Q1_new-Q1_old)/Q1_old)) <thread_inner and np.max(np.abs((pi1_new-pi1_old)/pi1_old)) <thread_inner:
      #print('Q1',il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)))
      break
  return Q1_new,pi1_new
sigma,_,xiterpolate = SigmaLambda(z_old,x_old,a1_old,theta23)
Q,pi = ValueIteration(theta1,sigma,xiterpolate)

def DataGeneration(theta23,theta1 = theta1,alpha = 1,T=TimePeriod,N=SamplePath,Nsub = Nsub,num_discrete=num_discrete):
  HdState = np.zeros([T,N],dtype= int)    #hidden state
  ObState = np.zeros([T,N],dtype = int) #observation state
  belief = np.zeros([T,N])                #belief x
  action1 = np.zeros([T,N],dtype = int)   #agent1's action

  HdState[0,:] = np.random.randint(0,2,size=N)
  belief[0,:] = np.kron(np.ones(Nsub),np.linspace(0,1,int(N/Nsub)))#np.random.uniform(size=N)
  ObState[0,:] = np.int_(np.random.random(size=N)>belief[0,:])
      

  trans,observe = Dynamic(theta23)
  sigma,_,xiterpolate= SigmaLambda(z_old,x_old,a1_old,theta23,num_discrete=num_discrete)
  _,pi1 = ValueIteration(theta1,sigma,xiterpolate,alpha=alpha)
  for t in range(T):
    # iterpolate belief
    belief_f = np.floor(belief[t,:] *(num_discrete-1))/(num_discrete-1)
    belief_c = np.ceil(belief[t,:] *(num_discrete-1))/(num_discrete-1)
    iterpolate = np.zeros(belief_f.shape)
    iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[t,:] -belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
    belief_fint,belief_cint = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

    # generate new action for both agents
    action_mat1 = np.random.random(size=N)
    p1 = iterpolate*pi1[belief_fint,0] + (1-iterpolate)*pi1[belief_cint,0]
    p2 = iterpolate*pi1[belief_fint,1] + (1-iterpolate)*pi1[belief_cint,1]
    action1[t,:] = np.int_(p1<=action_mat1) + np.int_(p2+p1<=action_mat1)
    
    if t!=T-1:
      # generate new hidden state and observation state
      state_mat1 = np.random.random(size=N)
      HdState[t+1,:] = np.int_(trans[HdState[t,:],action1[t,:],0]<=state_mat1)
      state_mat2 = np.random.random(size=N)
      ObState[t+1,:] = np.int_(observe[action1[t,:],HdState[t+1,:],0]<=state_mat2)

      # update new belief
      _,belief[t+1,:],_ = SigmaLambda(ObState[t+1,:] ,belief[t,:],action1[t,:],theta23,num_discrete=num_discrete,T=1)
  return ObState, action1,belief,HdState



# 0.02 Results

In [2]:
theta1,theta23

(array([-5, -1,  2]), array([[0.5   , 0.9999, 0.5   ],
        [0.5   , 0.85  , 0.5   ]]))

In [3]:
from tqdm import tqdm
from scipy.optimize import minimize
import pickle
lower_bound = 1e-6
# uper_bound = 1e4
ObState, action1,belief,HdState= DataGeneration(theta23,theta1=theta1.copy()*0.1,alpha=0.1)
print([np.sum(action1==i) for i in range(3)])



alpha_full = [1,0.1,0.01]
for alpha in alpha_full:
  print('-------------> alpha = ', alpha)
  ini_x0 = 0.85*theta23.reshape(-1)
  print('start from: ', ini_x0)
  def func1(theta23, ObState=ObState, action1=action1,x0=belief[0,:],T=TimePeriod):
    sigma,_,_ = SigmaLambda(ObState,x0,action1,theta23,num_discrete=num_discrete,T = TimePeriod)
    res = -np.sum(np.log(sigma))
    #print(theta23,res)
    return res#/TimePeriod/SamplePath
  res1 = minimize(func1,ini_x0, bounds=[[lower_bound,1-lower_bound]]*len(ini_x0))
  print(res1,res1.fun)#*TimePeriod*SamplePath)

  ini_x0 = alpha*theta1#(theta1[:,1:3].reshape(-1))#[0]
  print('start from: ', ini_x0)
  sigma,_,xiterpolate= SigmaLambda(z_old,x_old,a1_old,res1.x,num_discrete=num_discrete)
  _,_,beliefIP = SigmaLambda(ObState,belief[0,:],action1,res1.x,num_discrete=num_discrete,T = TimePeriod)
  def func2(theta1_est,sigma = sigma,xiterpolate=xiterpolate,beliefIP=beliefIP,action1 = action1,T=TimePeriod):
    # theta1_est = np.reshape(np.array([alpha*theta1[0,0],theta1_est[0],theta1_est[1],
    #                                   alpha*theta1[1,0],theta1_est[2],theta1_est[3]]),[2,3])
    _,pi1 = ValueIteration(theta1_est,sigma,xiterpolate,alpha=alpha)
    p1 = beliefIP[0] *pi1[beliefIP[1],action1] + (1-beliefIP[0]) *pi1[beliefIP[2],action1]
    #print(p1)
    res = - np.sum(np.log(p1)) 
    #print(theta1_est,res)
    return res#/TimePeriod/SamplePath
  cons= ({'type': 'ineq', 'fun': lambda x: lower_bound-x[0]},\
          {'type': 'ineq', 'fun': lambda x: lower_bound - x[1]},\
          {'type': 'ineq', 'fun': lambda x: x[2]-lower_bound})
  res2 = minimize(func2,ini_x0,bounds=[(-1e1*alpha,-1e-1*alpha),(-1e1*alpha,-1e-1*alpha),(1e-1*alpha,1e1*alpha)])#constraints=cons)#,bounds = [(-uper_bound,-lower_bound),(-uper_bound,-lower_bound),(lower_bound,uper_bound),
                              #          (lower_bound,uper_bound),(-uper_bound,-lower_bound),(-uper_bound,-lower_bound)])#method='Nelder-Mead')
  print(res2,res2.fun)#*TimePeriod*SamplePath)
  #print(res2.fun,res2.success,res2.x)

[211241, 312188, 226571]
-------------> alpha =  1
start from:  [0.425    0.849915 0.425    0.425    0.7225   0.425   ]
      fun: 498308.2893321474
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.13387762, -0.11059456, -0.27357601, -1.43772922,  1.51921995,
        0.13387762])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 147
      nit: 16
   status: 0
  success: True
        x: array([0.50008022, 0.99751507, 0.49097746, 0.4995251 , 0.84941752,
       0.49620917]) 498308.2893321474
start from:  [-5 -1  2]
      fun: 630864.6179196616
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.17462298, -0.17462298,  0.01164153])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 6
   status: 0
  success: True
        x: array([-4.943134  , -0.98437379,  1.97156001]) 630864.6179196616
-------------> alpha =  0.1
start from:  [0.425    0.849915 0.425    0.425    0.7225   0.425  

In [5]:
res1.fun+res2.fun

1129172.9072513054